In [ ]:
spark

Intitializing Scala interpreter ...

In [1]:
val df = spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9093")
  .option("subscribe", "kor_movies")
  .option("startingOffsets", "earliest")
  .load()

df.printSchema()
// complete 모드는 aggregate 연산이 있는 경우만 가능하므로 append 모드로 설정
val ds = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
  .as[(String, String)]
  .writeStream
  .outputMode("append")
  .queryName("debug")
  .format("console")
  .option("truncate", "false")
  .start()

// 모든 값들은 value 내에 포함된 json 을 타임에 맞게 읽어줘야 함

Intitializing Scala interpreter ...

Spark Web UI available at http://54cdf564c8c1:4041
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1657959006112)
SparkSession available as 'spark'


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



df: org.apache.spark.sql.DataFrame = [key: binary, value: binary ... 5 more fields]
ds: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@f8fbde9


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                              |
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|null|{"movies":{"movie":"movie","title":"title","title_eng":"title_eng","year":0,"grade":"grade"}}                                                                      |
|null|{"movies":{"movie":"10001","title":"시네마 천국","title_eng":"Cinema Paradiso , 1988","year":2013,"grade":"전체 관람가"}}                                          |
|null|{"movies":{"movie":"10002","title":"빽 투 더 퓨쳐","title

In [ ]:
ds.status

Intitializing Scala interpreter ...

In [28]:
ds.stop()

In [11]:
import org.apache.spark.sql.types._
// {"movie":"10010","title":"슈퍼맨 3","title_eng":"Superman III , 1983","year":0,"grade":"전체 관람가"}
val schema = StructType(
      StructField("movie", StringType, false) ::
      StructField("title", StringType, false) ::
      StructField("title_eng", StringType, false) ::
      StructField("year", IntegerType, true) ::
      StructField("grade", StringType, false) :: Nil)

import org.apache.spark.sql.types._
schema: org.apache.spark.sql.types.StructType = StructType(StructField(movie,StringType,false), StructField(title,StringType,false), StructField(title_eng,StringType,false), StructField(year,IntegerType,true), StructField(grade,StringType,false))


In [25]:
val movies = spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9093")
  .option("subscribe", "movies")
  .option("startingOffsets", "earliest")
  .load()

df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



movies: org.apache.spark.sql.DataFrame = [key: binary, value: binary ... 5 more fields]


In [26]:
val select = movies.select(col("key").cast("string"), from_json(col("value").cast("string"), schema).alias("movies"))

val sink = select
  .selectExpr("movies.title as title", "movies.year as year").as[(String, Int)]
  .writeStream
  .outputMode("append")
  .queryName("kafka-streaming")
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9093")
  .option("topioc", "kor_movies")
  .option("checkpointLocation", "/home/jovyan/work/data/tmp/kafka-sink")
  .start()

sink.awaitTermination()
sink.status

org.apache.spark.sql.streaming.StreamingQueryException:  Query kafka-streaming [id = 1d24ff78-619b-498e-9e22-ea3bf57bde66, runId = 6b96c306-83f7-4342-8c8d-87011dab5c8d] terminated with exception: org/apache/spark/sql/internal/connector/SupportsStreamingUpdate

In [ ]:
sink

In [12]:
val ds = select
  .writeStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9093")
  .option("topic", "kor_movies")
  .option("checkpointLocation", "/home/jovyan/work/data/tmp/scala/kafkasink")
  .start()

ds: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@99db418


In [22]:
println(ds.status)

{
  "message" : "Stopped",
  "isDataAvailable" : false,
  "isTriggerActive" : false
}


In [23]:
ds.awaitTermination()

In [27]:
ds.stop()